In [492]:
import torch
import time
from data import train_data_prepare
from train import train
from test import test, test_data_prepare
from model import Net
from model_baseline import BaselineNet
import os
os.environ["CUDA_VISIBLE_DEVICES"]="2"
import pandas as pd
import itertools
import liwc
import numpy as np

In [92]:

def read_dic(filepath):
    """
    Reads a LIWC lexicon from a file in the .dic format, returning a tuple of
    (lexicon, category_names), where:
    * `lexicon` is a dict mapping string patterns to lists of category names
    * `category_names` is a list of category names (as strings)
    """
    with open(filepath) as lines:
        # read up to first "%" (should be very first line of file)
        for line in lines:
            print(line)
            if line.strip() == "%":
                break
        # read categories (a mapping from integer string to category name)
        category_mapping = dict(_parse_categories(lines))
        # read lexicon (a mapping from matching string to a list of category names)
        lexicon = dict(_parse_lexicon(lines, category_mapping))
    return lexicon, list(category_mapping.values())




In [95]:
from liwc import Liwc
liwc = Liwc('LIWC2015_English_Flat.dic')
print(liwc.search('happy'))

ImportError: cannot import name 'Liwc'

In [94]:


import liwc
parse, category_names = liwc.load_token_parser('LIWC2015_English_Flat.dic')

import re

def tokenize(text):
    # you may want to use a smarter tokenizer
    for match in re.finditer(r'\w+', text, re.UNICODE):
        yield match.group(0)

gettysburg = '''Four score and seven years ago our fathers brought forth on
  this continent a new nation, conceived in liberty, and dedicated to the
  proposition that all men are created equal. Now we are engaged in a great
  civil war, testing whether that nation, or any nation so conceived and so
  dedicated, can long endure. We are met on a great battlefield of that war.
  We have come to dedicate a portion of that field, as a final resting place
  for those who here gave their lives that that nation might live. It is
  altogether fitting and proper that we should do this.'''.lower()
gettysburg_tokens = tokenize(gettysburg)



In [147]:
from tika import parser

raw = parser.from_file('LIWC2007 dictionary poster.pdf')
print(raw['content'])














































LIWC2007 dictionary poster.xlsx


Linguistic Inquiry and Word Count   LIWC 2007
NOTE: For research purposes only. This dictionary can only be used by non-commercial organizations such as schools or universities. Please do not share this dictionary with others outside your research lab.  Questions about commercial licenses should be directed to LIWCpermissions@receptiviti.com.

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 121 122 123 124 125 126 127 128 129 130 132 133 134 135 136 137 138 139 140 141 142 143 146 147 148 149 150 250 251 252 253 354 355 356 357 358 359 360 462 463 464
Funct Pronoun Ppron I We You SheHe They Ipron Article Verbs AuxVb Past Present Future Adverbs Prep Conj Negate Quant Numbers Swear Social Family Friends Humans Affect Posemo Negemo Anx Anger Sad Insight Cause Discrep Tentat Certain Inhib Incl Excl Percept See Hear Feel Bio Body Health Sexual Ingest Relativ Motion Space Time Work Achiev Leisure Home Mon

In [273]:
liwc_categories = raw['content'].split("\n")[52].split(" ")
liwc_categories

['Funct',
 'Pronoun',
 'Ppron',
 'I',
 'We',
 'You',
 'SheHe',
 'They',
 'Ipron',
 'Article',
 'Verbs',
 'AuxVb',
 'Past',
 'Present',
 'Future',
 'Adverbs',
 'Prep',
 'Conj',
 'Negate',
 'Quant',
 'Numbers',
 'Swear',
 'Social',
 'Family',
 'Friends',
 'Humans',
 'Affect',
 'Posemo',
 'Negemo',
 'Anx',
 'Anger',
 'Sad',
 'Insight',
 'Cause',
 'Discrep',
 'Tentat',
 'Certain',
 'Inhib',
 'Incl',
 'Excl',
 'Percept',
 'See',
 'Hear',
 'Feel',
 'Bio',
 'Body',
 'Health',
 'Sexual',
 'Ingest',
 'Relativ',
 'Motion',
 'Space',
 'Time',
 'Work',
 'Achiev',
 'Leisure',
 'Home',
 'Money',
 'Relig',
 'Death',
 'Assent',
 'Nonflu',
 'Filler']

In [399]:
def makeCatList(row_end,col_start,col_end,offset = 0):
    
    catlist = []
    idx = [(i,col_start,col_end) for i in range(startrow+offset,startrow+offset+row_end+1)]
    
    for (row,col_l,col_r) in idx:
        catlist.append(raw['content'].split("\n")[row].split(" ")[col_l:col_r])
    
    catlist = list(itertools.chain(*catlist))
    
    return catlist


list1 = makeCatList(160,0,3)
list2 = makeCatList(121,3,4)
list3 = makeCatList(73,4,5)
list4 = makeCatList(11,5,6)
list5 = makeCatList(11,6,7)
list6 = makeCatList(11,7,8)+makeCatList(8,5,6,len(list4))
list7 = makeCatList(11,8,9)+makeCatList(4,6,7,len(list5))
list8 = makeCatList(9,9,10)
list9 = makeCatList(9,10,11)+makeCatList(11,9,10)[-2:]


list9

makeCatList(80,4,5)[70:]


['disliked', 'dislikes', 'do', 'does', 'doesnt', "doesn't", 'doing']

In [448]:
startrow = 59
endrow = startrow + 161

array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])

In [513]:
includematrix = np.zeros((161, 105))*np.nan

In [665]:
#cat1
for i in range(160):
    includematrix[i,0]=1
    includematrix[i,1]=1
    includematrix[i,2]=1
    
includematrix[160,0]=1
includematrix[160,1]=1

#cat2
for i in range(121):
    includematrix[i,3]=1
    
#cat3
for i in range(73):
    includematrix[i,4]=1
    
#cat4
for i in range(11):
    includematrix[i,5]=1    
    
#cat5
for i in range(11):
    includematrix[i,6]=1   
    
#cat6
for i in range(20):
    includematrix[i,7]=1 
    
#cat7
for i in range(17):
    includematrix[i,8]=1  
    
#cat8
for i in range(9):
    includematrix[i,9]=1 
    
#cat9
for i in range(48):
    includematrix[i,10]=1 
    
#cat10
for i in range(3):
    includematrix[i,11]=1
    
#cat11
for i in range(133):
    includematrix[i,12]=1 
    includematrix[i,13]=1 
    includematrix[i,14]=1 

includematrix[133,12]=1
includematrix[133,13]=1


#cat12
for i in range(150):
    includematrix[i,15]=1 

#cat13
for i in range(151):
    includematrix[i,16]=1 
    
#cat14
for i in range(88):
    includematrix[i,17]=1 
    includematrix[i,18]=1 

includematrix[88,17]=1   

#cat 15
for i in range(50):
    includematrix[i,19]=1 
    

# cat 16
for i in range(72):
    includematrix[i,20]=1 
    

# cat 17
for i in range(62):
    includematrix[i,21]=1
    
# cat 18
for i in range(29):
    includematrix[i,22]=1
   

# cat 19
for i in range(59):
    includematrix[i,23]=1


# cat 20
for i in range(93):
    includematrix[i,24]=1
    

# cat 21
for i in range(35):
    includematrix[i,25]=1
    
    
# cat 22
for i in range(55):
    includematrix[i,26]=1
    
    
# cat 121
for i in range(157):
    includematrix[i,27]=1
    includematrix[i,28]=1
    includematrix[i,29]=1

includematrix[157,27]=1
includematrix[157,28]=1   
includematrix[158,28]=1  


# cat 122
for i in range(66):
    includematrix[i,30]=1
    
# cat 123
for i in range(38):
    includematrix[i,31]=1
    
# cat 124
for i in range(63):
    includematrix[i,32]=1
    
# cat 125
for i in range(156):
    includematrix[i,33]=1
    includematrix[i,34]=1
    includematrix[i,35]=1
    includematrix[i,36]=1
    includematrix[i,37]=1
    includematrix[i,38]=1
    
for i in range(159):
    includematrix[i,33]=1
    includematrix[i,34]=1
    includematrix[i,35]=1
    includematrix[i,36]=1
    includematrix[i,37]=1

includematrix[159,35]=1
includematrix[159,36]=1


# cat 126
for i in range(144):
    includematrix[i,38]=1
    includematrix[i,39]=1
    includematrix[i,40]=1
    
includematrix[144,38]=1
includematrix[144,39]=1
includematrix[145,38]=1
includematrix[145,39]=1
includematrix[146,39]=1
includematrix[147,39]=1




In [683]:
raw['content'].split("\n")[startrow+127].split(" ")

['few*',
 'part',
 'wholl',
 'guesses',
 "she's",
 "you'd",
 'whats',
 'used',
 'explain',
 'mum',
 "who'll",
 'confident',
 'fired',
 'improving',
 'pathetic*',
 'strange',
 'weird*',
 'fearless*',
 'merr*',
 'welcom*',
 'embarrass*',
 'low*',
 'sins',
 'worst',
 'cohere*',
 'found',
 'maybe',
 'rethink*',
 'whether',
 'sometimes',
 'noises',
 'weighting',
 'cuddl*',
 'hugg*',
 'pubic',
 'weigh',
 'closest',
 'further*',
 'noon*',
 'space',
 'yearly',
 'putting']

In [265]:
liwc_categories[0]



dict_1 = []
idx_1 = [(i,0,3) for i in range(startrow,startrow+160+1)]
idx_1.append((startrow+160+1,0,2))

for (row,col_l,col_r) in idx_1:
    dict_1.append(raw['content'].split("\n")[row].split(" ")[col_l:col_r])
    
dict_1 = list(itertools.chain(*dict_1))

In [275]:
dict_2 = []
idx_2 = [(i,3,4) for i in range(startrow,startrow+121+1)]

for (row,col_l,col_r) in idx_2:
    dict_2.append(raw['content'].split("\n")[row].split(" ")[col_l:col_r])
    
dict_2 = list(itertools.chain(*dict_2))

dict_2

['anybod*',
 'anyone*',
 'anything',
 'everybod*',
 'everyone*',
 'everything*',
 'hed',
 "he'd",
 'her',
 'hers',
 'herself',
 'hes',
 "he's",
 'him',
 '*',
 'his',
 'i',
 'Id',
 "I'd",
 "I'll",
 'Im',
 "I'm",
 'it',
 'itd',
 "it'd",
 "t'",
 "it'll",
 'its',
 "it's",
 'itself',
 'ive',
 "I've",
 'lets',
 "let's",
 'me',
 'mine',
 'my',
 'myself',
 'nobod*',
 'th',
 'other',
 'others',
 'our',
 'ours',
 'ourselves',
 'she',
 "she'd",
 "she'll",
 'shes',
 "she's",
 'somebod*',
 'someone*',
 'something*',
 'somewhere',
 'stuff',
 'that',
 'thatd',
 "that'd",
 'thatll',
 "that'll",
 'thats',
 "that's",
 'thee',
 'their*',
 'did',
 'themselves',
 'these',
 'they',
 'theyd',
 "they'd",
 'theyll',
 "they'll",
 'theyve',
 "they've",
 'thine',
 'thing*',
 'this',
 'those',
 'th',
 'thoust',
 'thy',
 'us',
 'we',
 "we'd",
 "we'll",
 "we're",
 'weve',
 "we've",
 'what',
 'whatever',
 'whats',
 "what's",
 'which',
 'whichever',
 'who',
 'whod',
 "who'd",
 'wholl',
 "who'll",
 'whom',
 'whose',
 '

In [210]:
print(raw['content'].split("\n")[startrow+160].split(" ")[0:3])
print(raw['content'].split("\n")[startrow+161].split(" ")[0:2])

['herself', 'shouldve', 'zillion*']
['hes', "should've"]


In [206]:
startrow = 59

startcol = 0
endcol = 125

print(raw['content'].split("\n")[startrow+160].split(" ")[startcol:endcol])

['herself', 'shouldve', 'zillion*', 'kindly', 'pride', 'wondered', 'throw*herself', 'shouldve', 'zillion*', 'kindly', 'pride', 'wondered', 'throw*']


In [184]:
print(raw['content'].split("\n")[startrow+160].split(" "))

['herself', 'shouldve', 'zillion*', 'kindly', 'pride', 'wondered', 'throw*herself', 'shouldve', 'zillion*', 'kindly', 'pride', 'wondered', 'throw*']


In [79]:
from collections import Counter
gettysburg_counts = Counter(category for token in gettysburg_tokens for category in parse(token))
print(gettysburg_counts)

Counter()


In [276]:
startrow

59

In [2]:
timestampTime = time.strftime("%H%M%S")
timestampDate = time.strftime("%d%m%Y")
timestampLaunch = timestampDate + '-' + timestampTime
parentPath = './models/'


#---Hyperparams
nnArchitecture = 'fake-net'
#lr = hyper['lr']
#epoch = hyper['epoch']
use_cuda = False
#num_classes = hyper['num_classes']

In [3]:
dataset_name = 'LIAR-PLUS'
train_filename = 'train2.tsv'
num_classes = 6
train_samples, word2num = train_data_prepare(train_filename, 2, dataset_name)

Preparing data from: train2.tsv
fault: 0
  10269 samples
  Statement Vocabulary Size: 1
  Subject Vocabulary Size: 1
  Speaker Vocabulary Size: 1
  Speaker Position Vocabulary Size: 1
  State Vocabulary Size: 1
  Party Vocabulary Size: 1
  Context Vocabulary Size: 1
  Justification Vocabulary Size: 1
  Vocabulary Size: 54832


In [4]:
label,
statement,
subject,
speaker,
speaker_pos,
state,
party,
context,
justification

NameError: name 'label' is not defined

In [5]:
train_file = open(train_filename, 'rb')
lines = train_file.read()
lines = lines.decode("utf-8")

table = []
for line in lines.strip().split('\n'):
    tmp = line.strip().split('\t')
    
    while len(tmp) < 16:
        tmp.append('')
    table.append([tmp[i].strip() for i in range(16)])

In [6]:
'''Column 1: the ID of the statement ([ID].json).
Column 2: the label.
Column 3: the statement.
Column 4: the subject(s).
Column 5: the speaker.
Column 6: the speaker's job title.
Column 7: the state info.
Column 8: the party affiliation.
Column 9-13: the total credit history count, including the current statement.
9: barely true counts.
10: false counts.
11: half true counts.
12: mostly true counts.
13: pants on fire counts.
Column 14: the context (venue / location of the speech or statement).

Column 15 extra: justification  '''

"Column 1: the ID of the statement ([ID].json).\nColumn 2: the label.\nColumn 3: the statement.\nColumn 4: the subject(s).\nColumn 5: the speaker.\nColumn 6: the speaker's job title.\nColumn 7: the state info.\nColumn 8: the party affiliation.\nColumn 9-13: the total credit history count, including the current statement.\n9: barely true counts.\n10: false counts.\n11: half true counts.\n12: mostly true counts.\n13: pants on fire counts.\nColumn 14: the context (venue / location of the speech or statement).\n\nColumn 15 extra: justification  "

In [7]:
df_table = pd.DataFrame(table)
df_table.columns = ["id","json_ID","label","statement","subjects","speaker","speaker_title","state","party",
                   "barely_true_counts","false_counts","half_true_counts","mostly_true_counts","pants_on_fire_counts",
                   "context","justification"]

df_table.head()

,id,json_ID,label,statement,subjects,speaker,speaker_title,state,party,barely_true_counts,false_counts,half_true_counts,mostly_true_counts,pants_on_fire_counts,context,justification
0,0,2635.json,false,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,0.0,0.0,0.0,a mailer,That's a premise that he fails to back up. Ann...
1,1,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.,"""Surovell said the decline of coal """"started w..."
2,2,324.json,mostly-true,"""Hillary Clinton agrees with John McCain """"by ...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver,"""Obama said he would have voted against the am..."
3,3,1123.json,false,Health care reform legislation is likely to ma...,health-care,blog-posting,,,none,7.0,19.0,3.0,5.0,44.0,a news release,"""The release may have a point that Mikulskis c..."
4,4,9028.json,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,,Florida,democrat,15.0,9.0,20.0,19.0,2.0,an interview on CNN,"""Crist said that the economic """"turnaround sta..."


In [47]:
df_table.query("speaker == 'barack-obama'  ")

,id,json_ID,label,statement,subjects,speaker,speaker_title,state,party,barely_true_counts,false_counts,half_true_counts,mostly_true_counts,pants_on_fire_counts,context,justification
2,2,324.json,mostly-true,"""Hillary Clinton agrees with John McCain """"by ...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver,"""Obama said he would have voted against the am..."
7,7,153.json,half-true,I'm the only person on this stage who has work...,ethics,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,"a Democratic debate in Philadelphia, Pa.","However, it was not that bill, but another one..."
16,16,620.json,true,McCain opposed a requirement that the governme...,federal-budget,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,a radio ad,"""I firmly object to all 'buy America' restrict..."
74,74,10039.json,mostly-true,The cost of health care is now the single-bigg...,"deficit,federal-budget,health-care",barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,a speech at Northwestern University,"""Obama saidthe cost of """"health care is now th..."
105,105,338.json,true,Sen. Obama has always had a 100 percent procho...,abortion,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,an e-mail message sent to voters before Super ...,"""The Planned Parenthood Action Fund similarly ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10166,10137,6423.json,half-true,Says Mitt Romney paid only 14 percent in taxes...,"candidates-biography,message-machine-2012,taxes",barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,a campaign ad,"Hoover, of course, was ushered out in 1932 as ..."
10187,10158,1094.json,half-true,"""The stimulus """"has delivered $43 billion in t...","economy,stimulus,taxes",barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,his weekly radio address,"His policy changes, from regulatory reform to ..."
10210,10181,625.json,barely-true,John McCain worries about nuclear waste in Ari...,energy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,a television ad,"""Patrick said: """"Texans pay the sixth-highest ..."
10221,10192,612.json,false,"""John McCain refuses to support a new bipartis...",energy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,a news release,"""We came the closest in 24 years to defeating ..."


### Speakers?

In [8]:
df_table.speaker.unique().shape

(2930,)

In [686]:
df_table.groupby("speaker").id.count().sort_values(ascending = False).iloc[120:190]

speaker
jim-moran                           12
jim-renacci                         12
will-weatherford                    12
saxby-chambliss                     11
randy-forbes                        11
                                    ..
greater-wisconsin-political-fund     8
allan-fung                           8
mark-neumann                         8
louie-gohmert                        8
michael-bloomberg                    8
Name: id, Length: 70, dtype: int64

In [687]:
df_table.query("speaker == 'allan-fung'  ")

,id,json_ID,label,statement,subjects,speaker,speaker_title,state,party,barely_true_counts,false_counts,half_true_counts,mostly_true_counts,pants_on_fire_counts,context,justification
1253,1253,9421.json,pants-fire,Ken Block supports Obamacare.,"economy,health-care,message-machine-2014",allan-fung,"mayor, city of Cranston, R.I.",Rhode Island,republican,2.0,2.0,0.0,2.0,1.0,a TV commercial,"""The very """"Newsmakers"""" appearance the Fung c..."
1959,1947,10051.json,false,"In the 38 Studios deal, Wall Street investors ...","bankruptcy,debt,economy,history,job-accomplish...",allan-fung,"mayor, city of Cranston, R.I.",Rhode Island,republican,2.0,2.0,0.0,2.0,1.0,a TV commercial,"""Nelson said, """"My state of Florida is now the..."
3971,3958,10172.json,barely-true,Gina Raimondo paid 70 million in taxpayer doll...,"candidates-biography,corporations,message-mach...",allan-fung,"mayor, city of Cranston, R.I.",Rhode Island,republican,2.0,2.0,0.0,2.0,1.0,a TV commercial,"Whats more, the average person does not view t..."
5220,5207,9562.json,mostly-true,Cranstons 2014-2015 budget funds 100 percent o...,"economy,income,job-accomplishments,labor,pensi...",allan-fung,"mayor, city of Cranston, R.I.",Rhode Island,republican,2.0,2.0,0.0,2.0,1.0,a letter to taxpayers,"""We believe in the middle. The independent res..."
5827,5814,9082.json,barely-true,"After hiring a campaign manager in 2006, I got...","economy,government-efficiency,jobs",allan-fung,"mayor, city of Cranston, R.I.",Rhode Island,republican,2.0,2.0,0.0,2.0,1.0,a radio interview,Oliver said the Keene Police Department cited ...
6953,6932,8527.json,mostly-true,We have created new jobs here in Cranston -- m...,"city-government,corporations,economy,governmen...",allan-fung,"mayor, city of Cranston, R.I.",Rhode Island,republican,2.0,2.0,0.0,2.0,1.0,a campaign speech,"Still, even if Todd Palin paid more for his sn..."
8294,8265,9810.json,false,Nazi imagery [was used] by the Block for Gover...,"elections,history,human-rights,campaign-advert...",allan-fung,"mayor, city of Cranston, R.I.",Rhode Island,republican,2.0,2.0,0.0,2.0,1.0,a news release,"""During the debate Ryan said that Obama was in..."
9829,9800,10129.json,true,Governor Carcieri has not been campaigning wit...,"candidates-biography,debates,campaign-advertising",allan-fung,"mayor, city of Cranston, R.I.",Rhode Island,republican,2.0,2.0,0.0,2.0,1.0,a Providence Journal-WPRI-12 debate,The number of flights has dropped from 240 to ...


In [696]:
import wikipedia

print(wikipedia.search("nancy-pelosi "))

['Nancy Pelosi', 'Alexandra Pelosi', 'Christine Pelosi', 'Paul Pelosi', 'Speaker of the United States House of Representatives', 'Susan Page', 'Jim Cramer', 'Ron Pelosi', 'Nancy Mace', 'Steny Hoyer']


In [694]:
print(wikipedia.suggest("nancy pelosi"))

None


In [692]:
print(wikipedia.page(wikipedia.search("nancy-pelosi ")[0]).links)

['100-Hour Plan', '100th United States Congress', '101st United States Congress', '102nd United States Congress', '103rd United States Congress', '104th United States Congress', '105th United States Congress', '106th United States Congress', '107th United States Congress', '108th United States Congress', '109th United States Congress', '110th United States Congress', '111th United States Congress', '112th Congress', '112th United States Congress', '113th United States Congress', '114th United States Congress', '115th United States Congress', '116th United States Congress', '117th United States Congress', '14th Dalai Lama', '1828 United States presidential election', '1832 Democratic National Convention', '1835 Democratic National Convention', '1840 Democratic National Convention', '1844 Democratic National Convention', '1848 Democratic National Convention', '1852 Democratic National Convention', '1856 Democratic National Convention', '1860 Democratic National Conventions', '1864 Democr

In [43]:
print(wikipedia.page('Nancy Pelosi').categories)

['1940 births', '20th-century American politicians', '20th-century American women politicians', '21st-century American politicians', '21st-century American women politicians', 'All BLP articles lacking sources', 'All Wikipedia articles written in American English', 'American people of Italian descent', 'American women activists', 'Articles containing video clips', 'Articles with Curlie links', 'Articles with short description', 'BLP articles lacking sources from January 2021', 'CS1 maint: extra text: authors list', 'California Democratic Party chairs', 'California Democrats', 'Catholics from California', 'Catholics from Maryland', 'Democratic Party members of the United States House of Representatives', 'Female members of the United States House of Representatives', 'Good articles', 'Italian-American culture in Baltimore', 'Italian-American culture in California', 'LGBT rights activists from the United States', 'Living people', 'Members of the United States House of Representatives fro

In [40]:
print(wikipedia.page('Nancy Pelosi').content)

Nancy Patricia Pelosi (; née D'Alesandro; born March 26, 1940) is an American politician serving as Speaker of the United States House of Representatives since 2019, and previously from 2007 to 2011. She has served as a U.S. representative from California since 1987. A member of the Democratic Party, Pelosi is the only woman in U.S. history to serve as speaker. She is second in the presidential line of succession, after vice president Kamala Harris. She is the dean of California's Congressional delegation.
Pelosi was first elected to Congress in 1987, following her father, Thomas D'Alesandro Jr., who served as a U.S. representative from Maryland and Mayor of Baltimore, into politics. She is the dean of California's congressional delegation, having begun her 18th term in 2021. Pelosi represents California's 12th congressional district, which comprises four-fifths of the city and county of San Francisco. She initially represented the 5th district (1987–1993), and then, when district boun